In [14]:
%matplotlib inline

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [23]:
conn = sqlite3.connect(':memory:')
c = conn.cursor()

In [24]:
c.execute('''
          CREATE TABLE IF NOT EXISTS USER
          ([user_id] INTEGER PRIMARY KEY, [product_name] TEXT,
          [name] varchar(255))
          ''')
          
c.execute('''
          CREATE TABLE IF NOT EXISTS ORDERS
          ([user_id] INTEGER , [amount] INTEGER,
          [order_id] INTEGER,[shop_id] INTEGER,
          [transaction_amount] INTEGER)
          ''')
                     
conn.commit()

In [25]:
c.execute('''
          INSERT INTO USER (user_id, name)

                VALUES
                (1,'Alan'),
                (2,'Barry'),
                (3,'Charlie'),
                (4,'Dave'),
                (5,'Ethan')
          ''')

c.execute('''
          INSERT INTO orders (user_id,order_id,shop_id, amount,transaction_amount)

                VALUES
                (1,1,90,8,800),
                (1,2,90,2,50),
                (3,3,89,1,10),
                (4,4,88,1,450),
                (5,5,87,3,150),
                (5,6,88,5,500)
          ''')

conn.commit()

In [37]:
query1 = c.execute('''select user.user_id, user.name,count(orders.order_id) from user
left join orders on user.user_id = orders.user_id
group by 1,2
order by 3 desc
''').fetchall()
query1_df = pd.DataFrame(query1, columns=['user_id','name','order_count'])
query1_df

,user_id,name,order_count
0,1,Alan,2
1,5,Ethan,2
2,3,Charlie,1
3,4,Dave,1
4,2,Barry,0


In [38]:
query2 = c.execute('''select user.user_id, user.name,count(orders.order_id) from user
left join orders on user.user_id = orders.user_id
group by 1,2
having count(orders.order_id) = 0''').fetchall()
query2_df = pd.DataFrame(query2, columns=['user_id','name','order_count'])
query2_df

,user_id,name,order_count
0,2,Barry,0


In [50]:
query2b = c.execute('''select user.user_id, user.name,count(orders.order_id) from user
left join orders on user.user_id = orders.user_id
where user.user_id not in (select user_id from orders)
''').fetchall()
query2b_df = pd.DataFrame(query2b, columns = ['user_id','name','order_count'])
query2b_df

,user_id,name,order_count
0,2,Barry,0


In [48]:
query3 = c.execute('''select user.user_id, user.name, count(orders.order_id)
from user
left join orders on user.user_id = orders.user_id
where orders.user_id in (select orders.user_id from orders where orders.transaction_amount > 100)
group by 1
''').fetchall()
query3_df = pd.DataFrame(query3, columns=['user_id','name','order_count_100_plus'])
query3_df

,user_id,name,order_count_100_plus
0,1,Alan,2
1,4,Dave,1
2,5,Ethan,2
